## Webscraping Hotpads 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.apartments.com/pittsburgh-pa'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}


In [3]:
# To get the html content from the url
html = requests.get(url, headers = headers)
htmlContent =  html.content

In [4]:
# The choice of parser - lxml or html.parser depends on the formatting of the html
# bsoup = BeautifulSoup(htmlContent, "lxml")
bsoup = BeautifulSoup(htmlContent, "html.parser")

In [5]:
pageInfo = bsoup.find('div',{'id':'paging'},{'class':'paging'}).findAll('a')
startPage = pageInfo[1].text
lastPage = pageInfo[len(pageInfo)-2].text
# lastPage 
# int(lastPage)


In [6]:
url1 = url+'/'+str(startPage)
html = requests.get(url1, headers = headers)
htmlContent = html.content

bSoup = BeautifulSoup(htmlContent,'html.parser')


In [7]:
placard_header = bSoup.find_all("header", {"class":"placardHeader"})
content = bSoup.findAll('div',{'class':'infoPadding'})


In [12]:
bSoup


<!DOCTYPE html>

<html data-placeholder-focus="false" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="email=no" name="format-detection"/>
<meta content="true" name="HandheldFriendly"/>
<title> Apartments for Rent in Pittsburgh PA - Page 28 | Apartments.com</title>
<link href="/a/8a55f9/favicon.ico" rel="icon"/>
<link href="https://www.apartments.com/pittsburgh-pa/28/" rel="canonical">
<link href="https://plus.google.com/+apartmentscom/" rel="publisher"/>
<meta content="See all 3,172 apartments in Pittsburgh, PA currently available for rent. Each Apartments.com listing has verified availability, rental rates, photos, floor plans and more." name="description"/>
<meta content="en" name="language"/>
<meta content="en" http-equiv="language"/>
<meta content="en-US" http-equiv="content-language"/>
<meta content="739299276

In [8]:
content

[<div class="infoPadding">
 <div class="freshnessAndFavoritesWrapper">
 <span class="listingFreshness">
 <span class="lastUpdated">
 <i class="rotateRightIcon" title="Last Updated"></i>
 <span>2 hrs Ago</span>
 </span>
 </span>
 <a class="favoriteIcon neutral" data-liked-classes="liked" data-neutral-classes="neutral" href="javascript:void(0);"></a> </div>
 <div class="apartmentRentRollupContainer">
 <span class="altRentDisplay">$1,259 - 2,978</span>
 <span class="unitLabel ">Studio - 2 Bed</span>
 <span class="availabilityDisplay noBedSelected ">Available Now</span>
 </div>
 <ul class="amenities">
 <li class="petIcon" title="Dogs"></li>
 <li class="catIcon" title="Cats"></li>
 <li class="airConditionerIcon" title="Air Conditioning"></li>
 <li class="laundryIcon" title="Washer/Dryer"></li>
 <li class="dishWasherIcon" title="Dishwasher"></li>
 </ul>
 <div class="contactInfo ">
 <div class="phone">
 <i class="phoneIcon"></i>
 <span>866-350-8444</span>
 </div>
 </div>
 <button class="check

In [13]:
web_content_list = []
for page in range(int(startPage),int(lastPage)+1):
    urlNew = url+'/'+str(page)
    html = requests.get(urlNew, headers = headers)
    htmlContent = html.content
    
    bSoup = BeautifulSoup(htmlContent, 'html.parser')
    
    placard_header = bSoup.find_all("header", {"class":"placardHeader"})
    content = bSoup.findAll('div',{'class':'infoPadding'})
#     address = bSoup.findAll('div',{'class':''})
    

    for item_header,item_content in zip(placard_header,content):  
        # To store the information to a dictionary
        web_content_dict = {}
        try:
            web_content_dict["ApartmentName"]=item_header.find("a",{"class":"placardTitle js-placardTitle "}).text.replace("\r","").replace("\n","")
            web_content_dict["Address"]=item_header.find("div",{"class":"location"}).text.replace("\r","").replace("\n","")
            web_content_dict["RentRange"] = item_content.find("span",{'class':'altRentDisplay'}).text
            web_content_dict["Description"] = item_content.find("span",{'class':'unitLabel'}).text
            web_content_dict["Phone"] = item_content.find("div",{'class':'phone'}).text.replace('\n',"")
        except:
            web_content_dict["ApartmentName"]=item_header.find("a",{"class":"placardTitle js-placardTitle perPerson"}).text.replace("\r","").replace("\n","")
            web_content_dict["Address"]=item_header.find("div",{"class":"location"}).text.replace("\r","").replace("\n","")
            web_content_dict["RentRange"] = item_content.find("span",{'class':'altRentDisplay'}).text
            web_content_dict["Description"] = item_content.find("span",{'class':'unitLabel'}).text
            web_content_dict["Phone"] = item_content.find("div",{'class':'phone'}).text.replace('\n',"")

        web_content_list.append(web_content_dict)
 
      

In [10]:
# web_content_list

In [16]:
df = pd.DataFrame(web_content_list)
df = df[['ApartmentName', 'Address', 'Description', 'Phone', 'RentRange']]
df


,ApartmentName,Address,Description,Phone,RentRange
0,City Vista,"325 Elizabeth Dr, Pittsburgh, PA 15220",Studio - 3 Bed,877-893-2069,"$994 - 4,476"
1,The Ridge at Robinson Apartments,"1501 Meredith Dr, Pittsburgh, PA 15205",1-3 Bed,412-533-5788,"$1,398 - 2,376"
2,Shadyside Commons,"401 Amberson Ave, Pittsburgh, PA 15232",1-2 Bed,412-208-4328,"$1,795 - 2,085"
3,The Gateway at Summerset,"1876 Parkview Blvd, Pittsburgh, PA 15217",1-2 Bed,844-228-9323,"$1,503 - 2,856"
4,Bakery Living Orange,"6350 Penn Ave, Pittsburgh, PA 15206",Studio - 2 Bed,844-814-2588,Call for Rent
5,Arsenal 201,"3922 Foster St, Pittsburgh, PA 15201",Studio - 3 Bed,844-831-9422,"$995 - 3,105"
6,Ollie at Baumhaus,"5522 Baum Blvd, Pittsburgh, PA 15232",Studio - 3 Bed,844-762-5514,"$1,035 - 5,325"
7,The Alden,"5492 Youngridge Dr, Pittsburgh, PA 15236",Studio - 3 Bed,412-532-4826,"$560 - 1,190"
8,Bryn Mawr,"100 Bryn Mawr Ct, Pittsburgh, PA 15221",1-2 Bed,888-863-7811,"$935 - 1,390"
9,The Ashby at South Hills Village Station,"1100 Village Dr, Pittsburgh, PA 15241",1-3 Bed,703-635-7163,"$1,075 - 2,575"
